# Exploratory Data Analysis

Aim: To analyze trends across multiple years of traffic collision datasets and confirm statistical significance.


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from pprint import pprint

<br>

## Data Acquisition

In [2]:
# Read all dataframes and store in a list
years: list[str] = ["2017", "2018", "2019", "2020", "2021"]

dfs: dict[str, pd.DataFrame] = {year: pd.read_csv(f"../usc_data/sc_unt{year}.csv", low_memory=False) for year in years}

In [3]:
# Length of each dataframe
for year, df in dfs.items():
    print(f"{year}: {len(df):,}")

2017: 268,999
2018: 269,752
2019: 268,227
2020: 224,591
2021: 274,401


In [4]:
# Check which features are available in each dataframe
columns: list[set[str]] = []
for df in dfs.values():
    columns.append(set(df.columns))  # Get all column names

print(f"Number of columns in each dataframe: {[len(columns) for columns in columns]}")

unique_cols: set[str] = columns[0].intersection(*columns)  # Intersection of all column names
print(f"Number of unique columns: {len(unique_cols)}")

Number of columns in each dataframe: [47, 47, 47, 47, 47]
Number of unique columns: 47


<br>

## Data Analysis
Since all columns appear in all dataframes, see the trend of each variable over the years
<br>Aim is to see if there is a trend in the data.

### Numeric Variable Analysis
Analyze trends for each NUMERIC variable.

In [5]:
# For each dataframe, add a 'year' column
for year, df in dfs.items():
    df['year'] = year

In [6]:
# Concatenate all dataframes
df: pd.DataFrame = pd.concat(dfs.values(), ignore_index=True)

In [7]:
# Check for missing values
print("\nMissing values:")
pprint(df.isnull().sum())


Missing values:
ano           0
aun           0
dsex          0
drac          0
dls       72360
dlc      108502
vmk       57041
rps        6378
vry       76716
cta           0
spl       24158
csn      963302
vlc1     965403
vlc2    1268401
dot           0
utc           0
vuc           0
vat           0
api           0
edp      993674
towd    1049925
edam          0
mhe           0
pd2     1028748
adi           0
cn2     1268089
tbr           0
man      610187
uor           0
atg     1253551
dtg     1256860
att     1288683
dtt     1299524
dtr     1302690
bdt     1292027
soe1          5
soe2    1073377
soe3    1214845
soe4    1274122
ecs       12864
ead        1896
fda         136
mda           0
atr2    1293125
vin      124745
noc           0
cdl           0
year          0
dtype: int64


In [8]:
# Analyze trends for each NUMERIC variable
numeric_cols: pd.Index = df.select_dtypes(include=[np.number]).columns
print(f"Number of numeric columns: {len(numeric_cols)}\n")

trends: dict[str, pd.Series] = {}  # Store mean of each numeric column for each year
for col in numeric_cols:
    trends[col] = df.groupby('year')[col].mean()
    
pd.DataFrame(trends)

Number of numeric columns: 27



,ano,spl,utc,vuc,api,edp,towd,edam,mhe,pd2,...,bdt,soe1,soe2,soe3,soe4,ead,fda,mda,atr2,noc
year,,,,,,,,,,,,,,,,,,,,,
2017,1.751230e+07,42.823821,9.882119,1.610820,7.565233,1702.516843,NaN,2.183384,24.407291,1270.515401,...,13.898116,20.971554,30.205857,38.677496,37.927545,2410.547117,9.860293,11.499745,0.123683,1.361960
2018,1.855833e+07,43.004679,10.028945,1.631806,7.536693,1782.653972,NaN,2.198523,24.529857,1688.594374,...,13.892883,20.975377,30.589912,39.066484,39.055884,2502.059501,9.651604,11.265588,0.122562,1.352665
2019,1.956362e+07,42.842381,10.183147,1.611974,7.471041,1997.934768,NaN,2.197430,24.360437,1648.004316,...,14.032791,21.002889,30.261866,38.812373,38.067112,2534.871199,9.600946,11.400526,0.123629,1.351840
2020,2.036490e+07,42.869119,9.913242,1.629967,7.333313,263.864218,0.475111,2.287126,24.972065,31.330312,...,17.195238,20.766260,31.556665,39.280714,39.240007,2800.612793,9.982207,12.240655,0.513581,1.308454
2021,2.138130e+07,43.258939,10.514346,1.626576,7.320141,200.647662,0.439986,2.291675,24.607636,31.004020,...,20.100394,20.891358,31.066628,39.272846,38.959001,2929.007906,9.995474,12.179970,0.543851,1.312382


In [15]:
# Count how many data points were removed for each variable in each year 
removal_counts: dict[str, pd.DataFrame] = {}

for col in numeric_cols:
    yearly_counts = df.groupby('year')[col].agg(['count', 'size'])
    yearly_counts['removed'] = yearly_counts['size'] - yearly_counts['count']
    removal_counts[col] = yearly_counts[['removed']]

In [17]:
# Perform statistical tests

# Store f-statistic and p-value for each numeric column
# Optionally, store any error that occurs during the test
results: dict[str, dict[str, float | str]] = {}

for col in numeric_cols:
    groups = [group[col].dropna().values for name, group in df.groupby('year')]
    groups = [group for group in groups if len(group) > 0]
    
    if len(groups) > 1:
        try:
            f_statistic, p_value = stats.f_oneway(*groups)
            results[col] = {
                'f_statistic': f_statistic, 
                'p_value': p_value, 
                'error': None,
                'removed_counts': removal_counts[col]
            }
        except Exception as e:
            results[col] = {
                'f_statistic': np.nan, 
                'p_value': np.nan, 
                'error': str(e),
                'removed_counts': removal_counts[col]
            }
    else:
        results[col] = {
            'f_statistic': np.nan, 
            'p_value': np.nan, 
            'error': 'Insufficient data for ANOVA',
            'removed_counts': removal_counts[col]
        }

In [21]:
print("\nRemoved data points:")
for col, result in results.items():
    print(f"\n{col}:")
    print(result['removed_counts'])


Removed data points:

ano:
      removed
year         
2017        0
2018        0
2019        0
2020        0
2021        0

spl:
      removed
year         
2017     4626
2018     5405
2019     5220
2020     3925
2021     4982

utc:
      removed
year         
2017        0
2018        0
2019        0
2020        0
2021        0

vuc:
      removed
year         
2017        0
2018        0
2019        0
2020        0
2021        0

api:
      removed
year         
2017        0
2018        0
2019        0
2020        0
2021        0

edp:
      removed
year         
2017   256798
2018   257227
2019   256377
2020   101747
2021   121525

towd:
      removed
year         
2017   268999
2018   269752
2019   268227
2020   113196
2021   129751

edam:
      removed
year         
2017        0
2018        0
2019        0
2020        0
2021        0

mhe:
      removed
year         
2017        0
2018        0
2019        0
2020        0
2021        0

pd2:
      removed
year         
2017  

In [22]:
# Visualize the results
pdf_filename = 'stats_results.pdf'
pdf = PdfPages(pdf_filename)  # Create a PdfPages object

for col, yearly_means in trends.items():
    plt.figure(figsize=(10, 6))
    yearly_means.plot(marker='o')
    plt.title(f'Trend of {col} over years')
    plt.xlabel('Year')
    plt.ylabel('Mean value')

    result = results[col]
    if result['error'] is None:
        annotation = f"f-statistic: {result['f_statistic']:.4f}\np-value: {result['p_value']:.4f}"
    else:
        annotation = f"Error: {result['error']}"
    
    plt.annotate(annotation, xy=(0.05, 0.95), xycoords='axes fraction', va='top')
    # plt.show()

    pdf.savefig()  # Save the current figure to the PDF
    plt.close()  # Close the current figure

pdf.close()  # Close the PdfPages object
print(f'All plots are saved to {pdf_filename}')

All plots are saved to stats_results.pdf
